# Read input files and define function

In [1]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import  make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression

In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle 

#==============================================================================
#  Define EMSC
#==============================================================================
#%% Extended multiplicative signal correction
def EMSC(X, reference, degree=4):
    # Create polynomials up to chosen degree
    poly = []; pvar = [1]
    for i in range(degree):
        poly.append( np.polyval(pvar,np.linspace(-1,1,len(reference))) )
        pvar.append(0)
    # Reference spectrum and polynomials
    emsc_basis = np.vstack([reference, np.vstack(poly)])
    # Estimate EMSC parameters
    (params,_,_,_) = np.linalg.lstsq(emsc_basis.T, X.T, rcond=None)
    # Correct and return
    return (X - params[1:,:].T @ emsc_basis[1:,:])/params[:1,:].T


#==============================================================================
#  Read train and test file
#==============================================================================
pickle_train = open("train.pkl","rb")
train_object = pickle.load(pickle_train)
pickle_test = open("test.pkl","rb")
test_object = pickle.load(pickle_test)

#==============================================================================
#  Read dictionary objects into arrays and Matrices
#==============================================================================
columns = train_object['shifts'].flatten()
X_train = train_object['RamanCal']
y_train = train_object['IodineCal']
replicates_train = train_object['repCal']

X_test = test_object['RamanVal']
replicates_test = test_object['repVal']

# Technical preprocessing

In [3]:
#==============================================================================
#  Keep only the shifts between 500 and 3100- train aand test
#==============================================================================
X_cut_train = X_train[ :, (columns>=500) & (columns<=3100)]
colnames = columns[ (columns>=500) & (columns<=3100) ]
X_emsc_train = EMSC(X_cut_train, X_cut_train[1343, :]  , degree=7)

X_cut_test = X_test[ :, (columns>=500) & (columns<=3100)]
X_emsc_test = EMSC(X_cut_test, X_cut_train[1343, :]  , degree=7)

# Creating DataFrames which are used later

In [4]:
#==============================================================================
#  Create dataframes for data visualization
#==============================================================================
col_str = list( map( str, colnames))
col_str.insert(0,'replicates')

test_df = pd.DataFrame( np.concatenate( (replicates_test[:, np.newaxis ]
            , X_emsc_test), axis =1), columns= col_str)

col_str.append('Iodine')

train_df = pd.DataFrame( np.concatenate( (replicates_train[:, np.newaxis ]
            , X_emsc_train, y_train), axis =1)
            , columns= col_str)

# PCA and RANSAC

In [7]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA, KernelPCA

from sklearn.linear_model import RANSACRegressor, LinearRegression

In [8]:
train_avg_df = train_df.iloc[:, :].groupby('replicates').mean()
test_avg_df = test_df.iloc[:, :].groupby('replicates').mean()

In [21]:
pip = make_pipeline(PCA(n_components=40), 
                    RANSACRegressor(
                        LinearRegression(), min_samples=100, max_trials=250, 
                        stop_score=0.995, random_state=0, loss='absolute_loss')
                    )

param_range  = np.arange(4, 30, 2)
param_range2 = np.arange(2, 2.5, 0.1)
param_grid   = [{'pca__n_components': param_range, 
                 'ransacregressor__residual_threshold': param_range2}]

gs = GridSearchCV(estimator=pip
                  , param_grid=param_grid
                  , cv=10, 
                  scoring='neg_mean_squared_error'
                 ,n_jobs=-1)

In [22]:
gs = gs.fit( train_avg_df.iloc[:,:-1].values , train_avg_df['Iodine'].values )

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [23]:
print('Best score:', gs.best_score_)
print(gs.best_params_)

Best score: -0.9655790845504355
{'pca__n_components': 14, 'ransacregressor__residual_threshold': 2.3}


In [24]:
output = pd.DataFrame( gs.predict( test_avg_df.iloc[:,:].values))
output['Id'] = output.index
output=output.rename(columns={ 0: "label"})
output.to_csv("avg_plsr_1.csv", index=False)

# More PLSR - second best prediction

In [166]:
train_avg_df = train_df.iloc[:, :].groupby('replicates').mean()
test_avg_df = test_df.iloc[:, :].groupby('replicates').mean()

In [167]:
comp_range = [ 14,15,16,17,18] 
scale_range = [True, False] 
max_iter_range = [10,20,50,100,500]

pls6 = make_pipeline( PLSRegression() )

param_grid = [ {'plsregression__n_components': comp_range
                , 'plsregression__scale': scale_range
                , 'plsregression__max_iter': max_iter_range} ]

gs = GridSearchCV(estimator=pls6,
                param_grid=param_grid,
                scoring='neg_mean_squared_error',
                cv=10,
                n_jobs=-1,
                refit=True,
                iid= False,
                  verbose=0)

In [168]:
gs = gs.fit( train_avg_df.iloc[:,:-1].values , train_avg_df['Iodine'].values )

In [169]:
print(gs.best_score_)
print(gs.best_params_)

-0.34293430892609794
{'plsregression__max_iter': 10, 'plsregression__n_components': 14, 'plsregression__scale': False}


In [170]:
output = pd.DataFrame( gs.predict( test_avg_df.iloc[:,:].values))
output['Id'] = output.index
output=output.rename(columns={ 0: "label"})
output.to_csv("avg_plsr_1.csv", index=False)

**MSE on the train dataset. Overfitting??**

In [171]:
mean_squared_error(  train_avg_df['Iodine'].values , gs.predict( train_avg_df.iloc[:,:-1].values )  )

0.052466510927491945